# Detailed Chemistry Modeling in PrOMMiS: Tutorial for Chemical Precipitation Applications
*A step-by-step guide for modeling aqueous reactions, pH control, and mineral scaling through integrating Reaktoro and PrOMMiS models*


---

## 1. Introduction

Chemical precipitation is a core mechanism in water treatment, hydrometallurgy, brine management, and critical mineral recovery. It describes the process in which dissolved ions combine to form a solid mineral phase once their activities exceed the solubility limit of that mineral.

In this tutorial, we cover a general workflow for:

- Modeling precipitation of various minerals 
- Adding reagents to control pH and alkalinity
- Predicting mineral scaling using Reaktoro model
- Setting up and solving a PrOMMis flowsheet model with updated property based on Reaktoro estimiation 




---

## 2. What Is Chemical Precipitation?

Chemical precipitation occurs when the **ionic activity product (IAP)** of certain ions exceeds a mineral’s **solubility product (Ksp)**:

$$
\text{IAP} = \prod_i a_i^{\nu_i},\qquad
\text{If IAP} > K_\text{sp},\ \text{mineral formation occurs.}
$$

Typical mineral categories for critical mineral and brine systems include:

- **Carbonates** (e.g., calcite, dolomite, siderite)  
- **Sulfates** (e.g., gypsum, barite)  
- **Hydroxides** (e.g., Mg(OH)₂, Fe(OH)₃)  
- **Mixed solid solutions** (e.g., transition-metal hydroxides or carbonates)

Reaktoro allows defining any number of aqueous species and solids from selected thermodynamic databases (e.g., PHREEQC, SUPCRT, EQ3/6). PrOMMis has the capability to develop a predictive model for the precipitation process. The integration of both capabilities inside the PrOMMis hub would enhance the existing PrOMMis model to consider detailed water chemistry.



---

## 3. Precipitation Processes Example Case

<img src="precipitation_pfd.jpg" alt="Chemical Precipitation PFD" width="80%">

In this example, the untreated wastewater would be treated by adding lime ($\text{CaO}$) to raise the $\text{pH}$ greater than 7, and calcite ($\text{CaCO}_3$) would be the precipitant in the sludge.

The inlet water composition is given below (concentrations in $\text{mg/L}$ as the ion):

| Ion | Concentration ($\text{mg/L}$) |
| :--- | :--- |
| $\text{Na}^+$ | 10556 |
| $\text{K}^+$ | 380 |
| $\text{Ca}^{2+}$ | 400 |
| $\text{Mg}^{2+}$ | 1272 |
| $\text{Cl}^-$ | 18980 |
| $\text{SO}_4^{2-}$ | 2649 |
| $\text{HCO}_3^-$ | 140 |


Add reagents **Sodium Carbonate** ($\text{Na}_2\text{CO}_3$) and **Calcium Oxide (Lime)** ($\text{CaO}$) as $\text{pH}$ adjusters, with doses of **1e-5 $\text{mol/L}$** and **1e-5 $\text{mol/L}$**, respectively.



# 4. Model Development for Precipitation Processes with Detailed Chemistry

This section details the development of the precipitation model. Our strategy involves a modular approach:

1.  **Develop the Detailed Chemistry Model (Section 4.1-4.7):** Use **Reaktoro-PSE** to accurately calculate chemical equilibrium, speciation, and mineral precipitation (using the Pitzer database) for the high-salinity feed stream.
2.  **Develop the Process Flowsheet Model (Section 4.8):** Define the unit operations using **PrOMMiS** components.
3.  **Integrate (Section 4.9 - 4.10):** Transfer the calculated chemistry parameters from Reaktoro into the PrOMMiS property model to achieve high-fidelity process simulation.

---

### 4.1. Import Modules from the Library

In [ ]:
from pyomo.environ import (
    Var,
    Param,
    Constraint,
    Expression,
    Objective,
    ConcreteModel,
    Block,
    value,
    assert_optimal_termination,
    units as pyunits,
)

from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import (
    set_scaling_factor,
    constraint_scaling_transform,
)
from idaes.core.util.model_statistics import degrees_of_freedom

from pyomo.util.calc_var_value import calculate_variable_from_constraint

# Import reaktoro-pse and reaktoro
from reaktoro_pse.reaktoro_block import ReaktoroBlock
import reaktoro
from reaktoro_pse.core.util_classes.cyipopt_solver import (
    get_cyipopt_watertap_solver,
)

### 4.2. Create Process Flowsheet Model to Begin the Model Development


In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

### 4.3. Create the Feed Brine Composition and Convert to Reaktoro-Enabled Format

In [ ]:
# Brine input assumptions in example case
brine_feed_composition = {
    "Na": 10556,
    "K": 380,
    "Ca": 400,
    "Mg": 1272,
    "Cl": 18980,
    "SO4": 2649,
    "HCO3": 140,
}
ion_mw = {
    "Na": 22.989769 / 1000,
    "K": 39.0983 / 1000,
    "Ca": 40.078 / 1000,
    "Mg": 24.305 / 1000,
    "Cl": 35.453 / 1000,
    "SO4": 96.06 / 1000,
    "HCO3": 61.0168 / 1000,
    "H2O": 18.01528 / 1000,
}
brine_feed_ph = 7.6

m.fs.feed = Block()
ions = list(brine_feed_composition.keys())
m.fs.feed.species_concentrations = Var(
    ions, initialize=1, bounds=(0, None), units=pyunits.mg / pyunits.L
)
ions.append("H2O")
m.fs.feed.species_mass_flow = Var(
    ions, initialize=1, bounds=(0, None), units=pyunits.kg / pyunits.s
)
m.fs.feed.specie_mw = Param(ions, units=pyunits.kg / pyunits.mol)
m.fs.feed.pH = Var(initialize=brine_feed_ph)
m.fs.feed.temperature = Var(initialize=293, units=pyunits.K)
m.fs.feed.pressure = Var(initialize=1e5, units=pyunits.Pa)
m.fs.feed.density = Var(initialize=1053, units=pyunits.kg / pyunits.m**3)

m.fs.feed.pH.fix()
m.fs.feed.temperature.fix()
m.fs.feed.pressure.fix()
m.fs.feed.density.fix()


# convert concentration to mass flows as Reaktoro-Enabled Format
@m.fs.feed.Constraint(list(m.fs.feed.species_mass_flow.keys()))
def eq_feed_species_mass_flow(fs, ion):
    if ion == "H2O":
        return Constraint.Skip
    else:
        return m.fs.feed.species_mass_flow[ion] == pyunits.convert(
            m.fs.feed.species_concentrations[ion]
            * m.fs.feed.species_mass_flow["H2O"]
            / m.fs.feed.density,
            to_units=pyunits.kg / pyunits.s,
        )

### 4.4. Add Precipitation Assumptions and Additional Constraints for Reaktoro Block Calculation

In [ ]:
m.fs.precipitation = Block()
m.fs.precipitation.reagents_mol_flow = Var(
    ["CaO", "Na2CO3"],
    initialize=1e-7,
    bounds=(1e-10, None),
    units=pyunits.mol / pyunits.s,
)

m.fs.precipitation.reagent_dose = Var(
    ["CaO", "Na2CO3"], initialize=1e-7, units=pyunits.mg / pyunits.kg
)
m.fs.precipitation.reagents_mw = Param(
    ["CaO", "Na2CO3"], units=pyunits.kg / pyunits.mol
)
m.fs.precipitation.reagents_mw["CaO"] = 56.0774 / 1000
m.fs.precipitation.reagents_mw["Na2CO3"] = 105.99 / 1000

m.fs.precipitation.precipitants = Var(["Calcite"], initialize=1e-32, units=pyunits.mol)
m.fs.precipitation.precipitants_mw = Param(
    ["Calcite"], initialize=100.09 / 1000, units=pyunits.kg / pyunits.mol
)

m.fs.precipitation.effluent_pH = Var(
    initialize=7, bounds=(5, 12), units=pyunits.dimensionless
)
m.fs.precipitation.effluent_alkalinity = Var(
    initialize=100, units=pyunits.mg / pyunits.L, bounds=(30, None)
)
m.fs.precipitation.effluent_species_mass_flows = Var(
    ions, initialize=1e-5, units=pyunits.kg / pyunits.s
)


@m.fs.precipitation.Constraint(ions)
def eq_effluent_species_mass_flows(fs, ion):
    if "Ca" == ion:
        return m.fs.precipitation.effluent_species_mass_flows["Ca"] == (
            m.fs.precipitation.reagents_mol_flow["CaO"] * m.fs.feed.specie_mw["Ca"]
            + m.fs.feed.species_mass_flow["Ca"]
            - m.fs.precipitation.precipitants["Calcite"] * m.fs.feed.specie_mw["Ca"]
        )
    elif "Na" == ion:
        return m.fs.precipitation.effluent_species_mass_flows["Na"] == (
            2
            * m.fs.precipitation.reagents_mol_flow["Na2CO3"]
            * m.fs.feed.specie_mw["Na"]
            + m.fs.feed.species_mass_flow["Na"]
        )
    elif "HCO3" == ion:
        return m.fs.precipitation.effluent_species_mass_flows["HCO3"] == (
            m.fs.feed.species_mass_flow["HCO3"]
            + m.fs.precipitation.reagents_mol_flow["Na2CO3"]
            * m.fs.feed.specie_mw["HCO3"]
            - m.fs.precipitation.precipitants["Calcite"] * m.fs.feed.specie_mw["HCO3"]
        )
    else:
        return (
            m.fs.precipitation.effluent_species_mass_flows[ion]
            == m.fs.feed.species_mass_flow[ion]
        )


@m.fs.precipitation.Constraint(["CaO", "Na2CO3"])
def eq_dose(fs, reagent):
    return m.fs.precipitation.reagent_dose[reagent] == pyunits.convert(
        m.fs.precipitation.reagents_mol_flow[reagent]
        * m.fs.precipitation.reagents_mw[reagent]
        / m.fs.feed.species_mass_flow["H2O"],
        to_units=pyunits.mg / pyunits.kg,
    )

### 4.5. Added Reaktoro Blocks into the Process for Detailed Chemistry Calculation

In [ ]:
m.fs.precipitation.reaktoro_outputs = {
    (
        "speciesAmount",
        "Calcite",
    ): m.fs.precipitation.precipitants["Calcite"],
    ("alkalinityAsCaCO3", None): m.fs.precipitation.effluent_alkalinity,
    ("pH", None): m.fs.precipitation.effluent_pH,
}
m.fs.precipitation.eq_precipitation = ReaktoroBlock(
    aqueous_phase={
        "composition": m.fs.feed.species_mass_flow,
        "convert_to_rkt_species": True,
        "activity_model": reaktoro.ActivityModelPitzer(),
        "fixed_solvent_specie": "H2O",
    },
    system_state={
        "temperature": m.fs.feed.temperature,
        "pressure": m.fs.feed.pressure,
        "pH": m.fs.feed.pH,
    },
    mineral_phase={"phase_components": ["Calcite"]},
    chemistry_modifier={
        "CaO": m.fs.precipitation.reagents_mol_flow["CaO"],
        "Na2CO3": m.fs.precipitation.reagents_mol_flow["Na2CO3"],
    },
    outputs=m.fs.precipitation.reaktoro_outputs,
    database_file="pitzer.dat",
    build_speciation_block=True,
)

### 4.6. Setting Scaling Factors, Initializing the Process, and Solving the Detailed Chemistry Estimation

In [ ]:
m.fs.feed.species_mass_flow["H2O"].fix(1.0)  # Fixing water flow to 1 kg/s
# fix concentrations
for ion, val in brine_feed_composition.items():
    m.fs.feed.species_concentrations[ion].fix(val)
    m.fs.feed.specie_mw[ion] = ion_mw[ion]
    set_scaling_factor(m.fs.feed.species_concentrations[ion], 1 / val)
m.fs.feed.specie_mw["H2O"] = ion_mw["H2O"]
for comp, pyoobj in m.fs.feed.eq_feed_species_mass_flow.items():
    calculate_variable_from_constraint(m.fs.feed.species_mass_flow[comp], pyoobj)
    set_scaling_factor(
        m.fs.feed.species_mass_flow[ion], 1 / m.fs.feed.species_mass_flow[comp].value
    )
    constraint_scaling_transform(pyoobj, 1 / m.fs.feed.species_mass_flow[comp].value)
set_scaling_factor(m.fs.feed.density, 1 / 1000)
set_scaling_factor(m.fs.feed.pH, 1)
set_scaling_factor(m.fs.feed.temperature, 1 / 273)
set_scaling_factor(m.fs.feed.pressure, 1e-5)
m.fs.precipitation.reagents_mol_flow["CaO"].fix(1e-2)
m.fs.precipitation.reagents_mol_flow["Na2CO3"].fix(1e-2)


for comp, pyoobj in m.fs.precipitation.eq_effluent_species_mass_flows.items():
    calculate_variable_from_constraint(
        m.fs.precipitation.effluent_species_mass_flows[comp], pyoobj
    )
    set_scaling_factor(
        m.fs.precipitation.effluent_species_mass_flows[ion],
        1 / m.fs.feed.species_mass_flow[comp].value,
    )
    constraint_scaling_transform(pyoobj, 1 / m.fs.feed.species_mass_flow[comp].value)

for reagent in m.fs.precipitation.reagents_mol_flow.keys():
    set_scaling_factor(m.fs.precipitation.reagents_mol_flow[reagent], 1e4)
    set_scaling_factor(m.fs.precipitation.reagent_dose[reagent], 1)
    constraint_scaling_transform(m.fs.precipitation.eq_dose[reagent], 1)
for reagent in m.fs.precipitation.precipitants.keys():
    set_scaling_factor(m.fs.precipitation.precipitants[reagent], 1e4)
set_scaling_factor(m.fs.precipitation.effluent_pH, 1)
m.fs.precipitation.eq_precipitation.initialize()

for key, obj in m.fs.precipitation.reaktoro_outputs.items():
    print(key, value(obj))

# recompute effluent composition using updated amount of formed calcite.
for comp, pyoobj in m.fs.precipitation.eq_effluent_species_mass_flows.items():
    calculate_variable_from_constraint(
        m.fs.precipitation.effluent_species_mass_flows[comp], pyoobj
    )

print(degrees_of_freedom(m))
assert degrees_of_freedom(m) == 0
solver = get_cyipopt_watertap_solver(max_iter=200, linear_solver="mumps", pivtol=1e-4)

result = solver.solve(m, tee=False)
assert_optimal_termination(result)

### 4.7. Display the Results

In [ ]:
lime_dose = value(m.fs.precipitation.reagent_dose["CaO"])
soda_ash_dose = value(m.fs.precipitation.reagent_dose["Na2CO3"])
effluent_ph = value(m.fs.precipitation.effluent_pH)
mass_in_ca = (
    value(m.fs.feed.species_mass_flow["Ca"])
    + value(m.fs.precipitation.reagents_mol_flow["CaO"])
    * value(m.fs.precipitation.reagents_mw["CaO"])
    * 0.7183
)

ca_removal = 100 * (
    1 - value(m.fs.precipitation.effluent_species_mass_flows["Ca"]) / mass_in_ca
)
hco3_removal = 100 * (
    (
        value(m.fs.feed.species_mass_flow["HCO3"])
        - value(m.fs.precipitation.effluent_species_mass_flows["HCO3"])
    )
    / value(m.fs.feed.species_mass_flow["HCO3"])
)
print("--- Results Summary ---")
print(f"Effluent pH: {effluent_ph:.2f}")
print("-" * 25)
print("\n--- Reagent Dosing ---")
print(f"Quicklime (CaO) Dose: {lime_dose:.3f} [mg/kg]")
print(f"Soda Ash (Na2CO3) Dose: {soda_ash_dose:.3f} [mg/kg]")
print("-" * 25)
print("\n--- Removal Efficiencies ---")
print(f"Calcium Removal: {ca_removal:.2f} %")
print(f"Bicarbonate/Alkalinity Removal: {hco3_removal:.2f} %")
print("-" * 25)

### 4.8 Create the PrOMMiS Process Model and Property Models with Example Case Assumptions and Preload Default Parameters Before Model Integration

In [ ]:
from idaes.core.initialization import BlockTriangularizationInitializer
from idaes.core.solvers import get_solver
from customized_liquid_properties import AqueousParameter
from customized_solids_properties import PrecipitateParameters
from prommis.precipitate.precipitator import Precipitator
import pyomo.environ as pyo


m.fs.properties_aq = AqueousParameter()
m.fs.properties_solid = PrecipitateParameters()

m.fs.unit = Precipitator(
    property_package_aqueous=m.fs.properties_aq,
    property_package_precipitate=m.fs.properties_solid,
)

m.fs.unit.aqueous_inlet.flow_vol[0].fix(10000)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "Na"].fix(10601.9)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "K"].fix(380)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "Ca"].fix(802.80)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "Mg"].fix(1272)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "Cl"].fix(18980)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "SO4"].fix(2649)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "HCO3"].fix(140)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "H2O"].fix(1000000)

m.fs.unit.cv_precipitate[0].temperature.fix(298.15)

assert degrees_of_freedom(m.fs.unit) == 0
initializer = BlockTriangularizationInitializer(constraint_tolerance=2e-5)
initializer.initialize(m.fs.unit)
solver = get_solver()
results = solver.solve(m.fs.unit, tee=True)
assert_optimal_termination(results)

### 4.8 Estimate the Parameters for Model Integration and Solve the Integrated Model

Integrate the Reaktoro Model and Prommis Model by Populating the Estimated Detailed Chemistry Parameters into the Property Model and Re-solve the Process Model


In [ ]:
m.fs.properties_aq.split["Ca"] = ca_removal
results = solver.solve(m.fs.unit, tee=True)

### 4.10 Display the Integrated Model Results



In [ ]:
print("\n---- Precipitate Metrics ----")
CaCO3_out = pyo.units.convert(
    m.fs.unit.precipitate_outlet.flow_mol_comp[0, "Ca(CO3)(s)"],
    to_units=pyo.units.mol / pyo.units.hr,
)
print(
    f"CaCO3 mol flow rate: "
    f"{pyo.value(CaCO3_out):.3g}"
    f"{pyo.units.get_units(CaCO3_out)}"
)